In [1]:
from os import chdir
chdir('..')

In [2]:
import matplotlib.pyplot as plt
from matplotlib.patches import Arc
import matplotlib
import math
import seaborn as sns
import pandas as pd
import numpy as np
from polylidar import extractPlanesAndPolygons
from shapely.geometry import LineString, asPolygon
from shapely.affinity import scale, translate
from os import path

from concave_evaluation.helpers import fake_building, plot_line, BLUE, GRAY, plot_arrow, scale_axes, plot_poly, load_polygon, get_max_bounds_polys, measure_convexity_simple
%matplotlib inline
sns.set(font_scale=1)  # crazy big

In [3]:
file_path_dir = './test_fixtures/results'
file_name = 'realsense_timings.csv'
df = pd.read_csv(path.join(file_path_dir, file_name))
df_reduced = df[['mean', 'l2','num_points' , 'alg', 'scene_name']]
df_reduced.head()

,mean,l2,num_points,alg,scene_name
0,50.894000,0.021905,85641,polylidar,Scene_001
1,249.484500,0.021771,85641,cgal,Scene_001
2,13170.829606,0.022257,85641,spatialite,Scene_001
3,2651.616716,0.081347,85641,postgis,Scene_001
4,49.797800,0.022875,85823,polylidar,Scene_002


In [4]:
df_reduced.groupby(['scene_name', 'alg']).mean().head(n=8)

mean        l2  num_points
scene_name alg                                           
Scene_001  cgal          249.484500  0.021771       85641
           polylidar      50.894000  0.021905       85641
           postgis      2651.616716  0.081347       85641
           spatialite  13170.829606  0.022257       85641
Scene_002  cgal          258.190700  0.022773       85823
           polylidar      49.797800  0.022875       85823
           postgis      2797.898841  0.083466       85823
           spatialite  16386.487627  0.023198       85823

## Aggregate Statistics

Aggregate statistics for all scenes

In [8]:
df_reduced.groupby(['alg']).agg(['mean', 'std', 'max'])

mean                                   l2            \
                    mean          std           max      mean       std   
alg                                                                       
cgal          248.357391    25.036466    267.735600  0.021589  0.014916   
polylidar      47.878854     4.315106     50.894000  0.021725  0.014975   
postgis      2734.730926   249.335923   2939.860773  0.075301  0.016628   
spatialite  13332.995400  2486.557941  16386.487627  0.021751  0.014782   

                        num_points                      
                 max          mean          std    max  
alg                                                     
cgal        0.064061  83184.545455  6531.783958  86026  
polylidar   0.064381  83184.545455  6531.783958  86026  
postgis     0.098799  83184.545455  6531.783958  86026  
spatialite  0.063373  83184.545455  6531.783958  86026